#Logistique

## Overview : Stocks of the 5 most ordered products (2023)

In [ ]:
SELECT p.productName, SUM(od.quantityOrdered) AS quant_vendue, ANY_VALUE(p.quantityInStock) AS Stock
FROM orderdetails od
INNER JOIN products p ON p.productCode = od.productCode
INNER JOIN orders o ON o.orderNumber = od.orderNumber
WHERE o.status = ""Shipped"" AND o.shippedDate LIKE ""2023-%""
GROUP BY p.productname
ORDER BY quant_vendue DESC LIMIT 5#(lf)

## Stock Unit

In [ ]:
SELECT p.productName, SUM(p.quantityInStock) AS quantite_stock
FROM products p
GROUP BY p.productName
ORDER BY p.productName;

## Stock Value Product Name

In [ ]:
SET sql_mode=(SELECT REPLACE(@@sql_mode,'ONLY_FULL_GROUP_BY',''));

SELECT productName, (quantityInStock*buyPrice) AS valeur_en_stock, quantityInStock AS nb_unite
FROM products
GROUP BY productName
ORDER BY valeur_en_stock DESC;

#Ressources Humaines

## Overview : Best seller per Month

In [ ]:
SELECT month_year, seller, turnover, profit, r.city
FROM (
	SELECT
   	 DATE_FORMAT(o.orderDate, "%Y %m") AS month_year,
   	 CONCAT(e.lastName, ' ', e.firstName) AS seller,
   	 SUM(od.quantityOrdered * od.priceEach) AS turnover,
   	 SUM(od.quantityOrdered * od.priceEach) - SUM(od.quantityOrdered * buyPrice) AS profit,
   	 oi.city,
    	RANK() OVER (PARTITION BY DATE_FORMAT(o.orderDate, "%Y %m") ORDER BY SUM(od.quantityOrdered * od.priceEach) DESC) AS seller_rank
	FROM orders o
	JOIN customers c ON c.customerNumber = o.customerNumber
	JOIN employees e ON e.employeeNumber = c.salesRepEmployeeNumber
	JOIN orderdetails od ON od.orderNumber = o.orderNumber
	JOIN products p ON p.productCode = od.productCode
	JOIN offices oi ON oi.officeCode = e.officeCode
	GROUP BY seller, month_year, oi.city
) AS r
WHERE seller_rank <= 2
ORDER BY month_year, seller_rank;

## Turnover per seller

In [ ]:
SELECT
    DATE_FORMAT(o.orderDate, "%Y %m") AS month_year,
    CONCAT(e.lastName, ' ', e.firstName) AS seller,
    SUM(od.quantityOrdered * od.priceEach) AS turnover,
    SUM(od.quantityOrdered * od.priceEach) - SUM(od.quantityOrdered * buyPrice) AS profit,
    oi.city
FROM orders o
JOIN customers c ON c.customerNumber = o.customerNumber
JOIN employees e ON e.employeeNumber = c.salesRepEmployeeNumber
JOIN orderdetails od ON od.orderNumber = o.orderNumber
JOIN products p ON p.productCode = od.productCode
JOIN offices oi ON oi.officeCode = e.officeCode
GROUP BY seller, month_year, oi.city
ORDER BY month_year, turnover DESC;

#Finance

## Overview and finance page : Turnover by country

In [ ]:
SET sql_mode=(SELECT REPLACE(@@sql_mode,'ONLY_FULL_GROUP_BY',''));

SELECT DATE_FORMAT(o.orderDate, '%Y-%m') AS Month, o.orderNumber, SUM(od.quantityOrdered * od.priceEach) AS ChiffreAffaire, SUM(od.quantityOrdered * p.buyPrice) AS CoutAchat
FROM orderdetails od
JOIN orders o ON o.orderNumber = od.orderNumber
JOIN products p ON p.productCode = od.productCode
WHERE o.status IN (""Shipped"" , ""Resolved"")
GROUP BY Month, od.orderNumber
ORDER BY od.orderNumber;

##Measures

In [ ]:
diffCAetMarge =
SUM('CAetMarge'[ChiffreAffaire]) - [Marge]

In [ ]:
Marge =
SUM('CAetMarge'[ChiffreAffaire]) - SUM('CAetMarge'[CoutAchat])

## Overview and finance page: Customer with payment default

In [ ]:
SELECT
    c.customerNumber AS numero_client,
    c.customerName AS nom_client,
    IFNULL(SUM(montant_commande), 0) AS montant_commande,
    IFNULL(SUM(paiements_recus), 0) AS paiements_recus
FROM
    customers c
LEFT JOIN (
    SELECT
        o.customerNumber,
        SUM(od.quantityOrdered * od.priceEach) AS montant_commande
    FROM
        orders o
    JOIN
        orderdetails od ON o.orderNumber = od.orderNumber
    GROUP BY
        o.customerNumber
) AS cmd ON c.customerNumber = cmd.customerNumber

LEFT JOIN (
    SELECT
        p.customerNumber,
        SUM(p.amount) AS paiements_recus
    FROM
        payments p
    GROUP BY
        p.customerNumber
) AS pay ON c.customerNumber = pay.customerNumber

GROUP BY
    c.customerNumber, c.customerName
ORDER BY
    c.customerNumber;

##Measure :

In [ ]:
ResteApayer = 'Requête1'[montant_commande]-'Requête1'[paiements_recus]

# Ventes

## Overview and Sales page

In [ ]:
SELECT productlines.productline AS categories,
DATE_FORMAT(orderdate, '%Y-%m') AS annee_mois,
SUM(orderdetails.quantityordered) AS nb_produits_commandes
FROM products
INNER JOIN orderdetails ON products.productcode=orderdetails.productcode
INNER JOIN productlines ON products.productline=productlines.productline
INNER JOIN orders ON orderdetails.ordernumber=orders.ordernumber
GROUP BY categories, DATE_FORMAT(orderdate, '%Y-%m')
ORDER BY categories ASC, annee_mois DESC;

## measure : rate of change

In [ ]:
Rate of change (%) with Y - 1 =
IF( ISFILTERED('TestSales'[nb_produits_commandes]),
	ERROR("error"),
	VAR PREVIOUS_YEAR =
		CALCULATE(
			SUM('TestSales'[nb_produits_commandes]),
			DATEADD('TestSales'[annee_mois].[Date], -1, YEAR)
		)
	RETURN
		DIVIDE(
			SUM('TestSales'[nb_produits_commandes]) - PREVIOUS_YEAR,
			PREVIOUS_YEAR
		)
)